<img src="TuneMyMoodLogo.png">

O TuneMyMood é um projeto da matéria de Ciência dos Dados feito por alunos do segundo semestre de Engenharia de Computação no Insper. O projeto consiste em utilizar de classificadores, no caso Naive-Bayes e Supported Vector Classification(Classificação de Suporte Vetorial), para conseguir separar músicas por gênero musical e moods(humores) com base nos dados, recolhidos pelo Spotipy, uma API do serviço de streaming de música Spotify, de: 
- acousticness(indice de 0 a 1 que representa o quão acústica é uma música), 
- danceability(índice de 0 a 1 que representa o quão boa uma música é para dançar), 
- instrumentalness(íncide de 0 a 1 que representa o quão próxima uma música chega de ser totalmente intrumental, sendo que sons como "Ooh" e "Aah" são considerados instrumentais), 
- key(Representa o tom na qual a música é tocada, começando da seguinte maneira: Dó=1, Ré=2, Mi=3, etc.), 
- liveness(Índice de 0 a 1 que representa o quão ao vivo uma música é, baseada em ruídos externos), 
- loudness(Índice de 0 a 1 que representa o quão barulhenta uma música é), 
- mode(Representa tons maiores ou menores. Exemplo: Dó maior = 1, Ré menor = 0), 
- speechiness(Índice de 0 a 1 que representa o quanto da música são palavras faladas ao invés de cantadas), 
- tempo(Representa o número de batidas por minuto da música, ou em outras palavras, o quão rápida ela é), 
- time_signature(Índice que representa o compasso da música, fração que representa o agrupamento de sons de uma música. Estilos diferentes costumam ter compassos específicos, como o compasso na valsa que é 3/4), 
- valence(Índice de 0 a 1 que representa o humor de uma música, sendo 0 uma música triste e 1 uma feliz)

### Índice
<ul>
<li><a href=#api> 1.0 Utilizando a API Spotipy</a></li>
<li><a href=#classificadores> 2.0 Entendendo nossos classificadores</a></li>
<li><a href=#nb> 2.1 Classificador de Naive-Bayes</a></li>
<li><a href=#svc> 2.2 SVC - Classificador por Vetores de Suporte</a></li>
<li><a href=#train> 3.0 Treinando nossos classificadores</a></li>
<li><a href=#dataset> 4.0 Classificando um Dataset</a></li>
<li><a href=#versus> 5.0 Comparação: Naive-Bayes VS SVC</a></li>
<li><a href=#melhorias> 6.0 Possíveis melhorias ao modelo</a></li>
</ul>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.svm import SVC
from tmm_functions import *

<div id="api"><div/>
### Utilização da API Spotipy
Ao linkar a API à um Client_User e um User_Secret, que são distribuidos pelo Spotify para quem quer fazer uso de uma API referente à ele, podemos baixar uma playlist a partir de seu playlist_id e seu user_id, ambos coletados a partir de uma url. Como exemplo de seu funcionamento, vamos baixar duas playlists de Rock e ver como elas são retornadas pelo Spotipy

In [2]:
lista_de_urls = ["https://open.spotify.com/user/spotify/playlist/37i9dQZF1DX153gOfbCM2i",
                "https://open.spotify.com/user/spotify/playlist/37i9dQZF1DX49jUV2NfGku"]



def lister(listaurl): #Função que recebe a lista de urls e devolve elas em uma lista de listas de
    liste = []        #["nome","user_id","playlist_id"]
    
    for i in range(len(listaurl)):
        a = re.split(r"/", listaurl[0])
        temp = [a[5]+str(i+1),a[4],a[6]]
        liste.append(temp)
    return liste



lista_pronta = lister(lista_de_urls)
with open("login.txt") as f: #Abrindo o login.txt que contem na 1a linha Client_user e na segunda a Client_secre
    data = f.readlines()

client=User(data) #Pegando os dados de um aqruivo .txt que está escondido com Gitignore


def list_download(list_urls,lista_generos,genre,datavalue): #Função que baixa a playlist usando Spotipy em Dataframe
    
    #Note que já definiremos o gênero musical, já com classificação de Sim(1) ou Não(0)
    
    #list_urls = lista de urls
    #lista_generos = lista de generos - Facilita para ja marcar outros generos como Não(0)
    #genre = genero da playlist
    #datavalue = Se é ou não desse genero - Sim(1) ou Não(0)
    
    df_baixada = client.playlist_downloader(list_urls[0][0],list_urls[0][1],list_urls[0][2])
    print(1)
    df_list = []
    for i in range(1,len(list_urls)):
        df =client.playlist_downloader(list_urls[i][0],list_urls[i][1],list_urls[i][2])
        df_baixada = pd.concat([df_baixada,df])
        print(i+1)
    for i in range(len(lista_generos)):
        df_baixada[lista_generos[i]] = 0
    if datavalue == 1:
        df_baixada[genre] = datavalue
    print("SO DONE")
    return df_baixada #Playlist retornada em Dataframe

lista_generos=['funk','rock','eletronica','metal','sertanejo','rap']

df_spotipy_test = list_download(lista_pronta,lista_generos,"rock",1) #Todos os valores "Rock" serão fornecidos como 1
df_spotipy_test.head()

1
2
SO DONE


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,uri,valence,song_title,artist,funk,rock,eletronica,metal,sertanejo,rap
0,0.004610,https://api.spotify.com/v1/audio-analysis/6rUp...,0.498,231480,0.830,6rUp7v3l8yC4TKxAAR5Bmx,0.000000,6,0.139,-5.157,...,spotify:track:6rUp7v3l8yC4TKxAAR5Bmx,0.453,I Hate Everything About You,Three Days Grace,0,1,0,0,0,0
1,0.000353,https://api.spotify.com/v1/audio-analysis/7kwi...,0.420,210240,0.929,7kwiwQhWRBSP1PJv4ZFJUK,0.000747,7,0.122,-3.899,...,spotify:track:7kwiwQhWRBSP1PJv4ZFJUK,0.300,Chop Suey!,System Of A Down,0,1,0,0,0,0
2,0.006240,https://api.spotify.com/v1/audio-analysis/6ZOB...,0.545,233827,0.860,6ZOBP3NvffbU4SZcrnt1k6,0.000022,11,0.165,-5.054,...,spotify:track:6ZOBP3NvffbU4SZcrnt1k6,0.581,Kryptonite,3 Doors Down,0,1,0,0,0,0
3,0.000172,https://api.spotify.com/v1/audio-analysis/1AEY...,0.492,285570,0.896,1AEYT6VxrxXPMoQUxsY0E4,0.000003,4,0.189,-5.819,...,spotify:track:1AEYT6VxrxXPMoQUxsY0E4,0.534,Headstrong,Trapt,0,1,0,0,0,0
4,0.001400,https://api.spotify.com/v1/audio-analysis/1p1n...,0.309,231533,0.912,1p1nO35bbi4ZlQgjIA4oa4,0.000271,4,0.582,-3.881,...,spotify:track:1p1nO35bbi4ZlQgjIA4oa4,0.302,The Kill (Bury Me),Thirty Seconds To Mars,0,1,0,0,0,0


Perceba que as últimas colunas são os gêneros musicais. Na lógica utilizada no trabalho, uma música comtempla apenas um estilo musical, sendo isso uma limitação do nosso modelo, uma vez que sabemos que existem músicas que se enquadram em 2 ou mais estilos.

<div id="classificadores"></div>
### Entendendo nosso classificadores

#### Classificadores
Classificadores são ferramentas que quando treinadas conseguem tentar adivinhar qual será o output de um input. Isso quer dizer que você as treina inserindo um input e lhe dizendo o output desejado, de forma que você tentará depois utilizar esse classificador para adivinhar o output de outros inputs. Um exemplo bem vísivel seria o seguinte: Imagine que eu quero treinar um computador para indentificar fotos de gatos. Eu lhe mostro algumas fotos de gatos e digo "isso é um gato", depois mostro fotos de coisas que não são gatos, como cachorros, cavalos ou outras coisas e lhe digo "isso não é um gato", com base o input(a foto) e o output(o que lhe disse) ele é treinado para quando eu lhe mostrar uma foto, ele tentar adivinhar se aquilo é um gato ou não. CLassificadores mais complexos já conseguem receber outputs mais complicados simples booleanos como esse, tornando o Machine Learning mais complexo e eficiente. No nosso caso vamos usar booleanos, pois cumprem muito bem o nosso propósito.

<div id="nb"></div>
#### Naive Bayes
O classificador de Naive-Bayes cálcula duas probabilidades, uma é a probabilidade do input ser Sim(1) e a outra de ser Não(0). Ele então compara as duas e a maior é considerada a classificação real. O cálculo observa os termos(como tempo, valence, acousticness) como independentes. A conta de cada probabilidade pode ser mostrada pelo exemplo:

Sendo A um objeto com 3 termos: A1, A2, A3.

P(Sim/A) = P(A1/Sim)xP(A2/Sim)xP(A3/Sim)XP(Sim)
                            P(A)
                            
Note que P(An/Sim) pode ser definido por { CountAn / TotalSim }, ou seja, quantas vezes esse valores do termo A1 apareceu no treino em Sim. Fora isso, como haverá uma Comparação entre P(Sim/A) e P(Não/A), não precisamos calcular o P(A), pois será igual aos dois e o que import é qual é maior.

Dessa forma, se P(Sim/A) > P(Não/A), A = 1, se <, A = 0
<div id="svc"></div>
#### Support Vector Classificator
Support Vector Machines são poderosos algoritmos para tanto regressão quanto classificação. No caso do classificador(SVC), usamos o que é chamado de classificação discriminativa, ou seja, ao invés de modelar cada classe(como no NB) o SVC visualiza um plano cartesiano e traça uma linha ou curva que consegue separar os dados, da seguinte maneira:

Olhando de forma gráfica, vemos que queremos criar uma lógica que consiga dividir quaisquer valores que forem adicionados em 1 ou 0. Observando um plano simples(2D), com os termos A1 e A2 nos eixos, podemos observar que SVC é uma forma de classificar:

<img src="graph1.png">

A função do SVC é dividir esse gráfico em duas partes, por meio de uma reta ou curva:

<img src="graph2.png">
Tudo que estiver acima da linha verde será classificado como 0 e a abaixo como 1.

Porém, existem casos onde uma linha não cumprirá sua função, e por isso o SVC consegue utilizar de curvas também:
<img src="graph3.png">

O SVC consegue fazer isso para diversos resultados também:
<img src="graph4.png">

O classificador vetorial trabalha com diversos termos, não apenas dois, mas por questão de visualização utilizamos apenas 2 nesses exemplos.

<div id="train"></div>
### Treinando nossos classificadores
Agora que entendemos o funcionamento dessas ferramentas, vamos utilizá-las a partir da biblioteca scikit-learn para classificar nossos dados. As facilidades dessa biblioteca são inumeras, mas uma delas é o fator de podermos usar praticamente o mesmo código para o NB quanto para o SVC, como poderá ser observado no código a seguir. Além disso, não vamos usar as duas playlists coletadas nesse código para o classificador,pois seriam apenas cerca de 120 músicas, tal que isso seria um número pequeno para um treino preciso, então coletamos(vide: DataCollector.ipynb) cerca de 600 músicas(10 playlists) de cada gênero. 
Então para, por exemplo, Rock, vamos utilizar um rock_df, apanhado de 600 musicas de Rock, como 1 e um not_rock_df, um apanhado de 600 musicas de variados estilos sem Rock, como 0.

In [3]:
class Dataset: #classe para formatação e analise de datasets
    
    def __init__(self,df2): #df será passado como string do diretorio do csv
        if type(df2)==str:
            self.df=pd.read_csv(df2)
            self.df["loudness"]=abs(self.df["loudness"])
            self.dict=self.df.filter(items=['acousticness','danceability','instrumentalness','key','liveness','loudness','mode'
                      ,'speechiness','tempo','time_signature','valence','artist']).to_dict('records')
        else:   
            self.df=df2
            self.df["loudness"]=abs(self.df["loudness"])
            self.dict=self.df.filter(items=['acousticness','danceability','instrumentalness','key','liveness','loudness','mode'
                      ,'speechiness','tempo','time_signature','valence','artist']).to_dict('records')
            
        self.ans=self.df[self.df.columns[-1]].tolist()
        self.vec=DictVectorizer()
        self.train_songs=self.vec.fit_transform(self.dict).toarray()

# Abrindo Dataframes De Gênero
funk_df = pd.read_csv("funk_df.csv")
rock_df = pd.read_csv("rock_df.csv")
eletronica_df = pd.read_csv("eletronica_df.csv")
metal_df = pd.read_csv("metal_df.csv")
sertanejo_df = pd.read_csv("sertanejo_df.csv")
rap_df = pd.read_csv("rap_df.csv")

# Abrindo Dataframes De Não Gênero
not_funk_df = pd.read_csv("not_funk_df.csv")
not_rock_df = pd.read_csv("not_rock_df.csv")
not_eletronica_df = pd.read_csv("not_eletronica_df.csv")
not_metal_df = pd.read_csv("not_metal_df.csv")
not_sertanejo_df = pd.read_csv("not_sertanejo_df.csv")
not_rap_df = pd.read_csv("not_rap_df.csv")

# Organizando na Classe Dataset
funk_df = Dataset(funk_df)
rock_df = Dataset(rock_df)
eletronica_df = Dataset(eletronica_df)
metal_df = Dataset(metal_df)
sertanejo_df = Dataset(sertanejo_df)
rap_df = Dataset(rap_df)
not_funk_df = Dataset(not_funk_df)
not_rock_df = Dataset(not_rock_df)
not_eletronica_df = Dataset(not_eletronica_df)
not_metal_df = Dataset(not_metal_df)
not_sertanejo_df = Dataset(not_sertanejo_df)
not_rap_df = Dataset(not_rap_df)



In [4]:
df_t = not_rap_df.df.drop_duplicates()
df_t

,Unnamed: 0,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,...,uri,valence,song_title,artist,rock,funk,eletronica,rap,sertanejo,metal
0,0,0.000721,https://api.spotify.com/v1/audio-analysis/01IC...,0.539,515387,0.828,01ICFoBoco3TaM5r3HthXy,0.438000,4,0.1980,...,spotify:track:01ICFoBoco3TaM5r3HthXy,0.5870,Master Of Puppets,Metallica,1,0,0,0,0,0
1,1,0.017600,https://api.spotify.com/v1/audio-analysis/7oGW...,0.358,314373,0.806,7oGWqVwmgwGBYXkW4cROlK,0.002520,7,0.0982,...,spotify:track:7oGWqVwmgwGBYXkW4cROlK,0.5780,Over and Over Again,The Used,1,0,0,0,0,0
2,2,0.001840,https://api.spotify.com/v1/audio-analysis/7nAf...,0.593,231307,0.712,7nAfXgeHfDO50upcOjJOaq,0.000000,2,0.1280,...,spotify:track:7nAfXgeHfDO50upcOjJOaq,0.4730,Talking to Myself,Linkin Park,1,0,0,0,0,0
3,3,0.000115,https://api.spotify.com/v1/audio-analysis/4U9a...,0.494,188228,0.778,4U9arGcc4ATa7shChhLQP0,0.000000,8,0.0904,...,spotify:track:4U9arGcc4ATa7shChhLQP0,0.4060,Walk On Water,Thirty Seconds To Mars,1,0,0,0,0,0
4,4,0.000209,https://api.spotify.com/v1/audio-analysis/6JSr...,0.515,266187,0.917,6JSryEdVJTZq6YBn3wK2sn,0.000476,1,0.1450,...,spotify:track:6JSryEdVJTZq6YBn3wK2sn,0.4090,Arrows,Foo Fighters,1,0,0,0,0,0
5,5,0.047700,https://api.spotify.com/v1/audio-analysis/54Ra...,0.367,281781,0.903,54RaEDwHbCJKMdDq68EyKE,0.001240,4,0.2350,...,spotify:track:54RaEDwHbCJKMdDq68EyKE,0.2390,The Doomed,A Perfect Circle,1,0,0,0,0,0
6,6,0.000006,https://api.spotify.com/v1/audio-analysis/4moZ...,0.431,234880,0.975,4moZ8avSDXYdJhMYwL60kW,0.667000,1,0.5520,...,spotify:track:4moZ8avSDXYdJhMYwL60kW,0.7150,Holy Mountain,Noel Gallagher's High Flying Birds,1,0,0,0,0,0
7,7,0.001770,https://api.spotify.com/v1/audio-analysis/51tx...,0.568,197933,0.871,51txLjbvSBRvtxjCjAFLkM,0.000406,1,0.3600,...,spotify:track:51txLjbvSBRvtxjCjAFLkM,0.6930,How Did We Get So Dark?,Royal Blood,1,0,0,0,0,0
8,8,0.002570,https://api.spotify.com/v1/audio-analysis/45HA...,0.447,193714,0.788,45HAjqRWiNv6mMPw4NvZrU,0.000017,2,0.0744,...,spotify:track:45HAjqRWiNv6mMPw4NvZrU,0.5050,Come Together,Gary Clark Jr.,1,0,0,0,0,0
9,9,0.037900,https://api.spotify.com/v1/audio-analysis/2jA5...,0.562,202507,0.588,2jA5Vp93KvnH0tXN6Fpwvl,0.000048,3,0.0663,...,spotify:track:2jA5Vp93KvnH0tXN6Fpwvl,0.6500,Spray Paint Love,Frank Carter & The Rattlesnakes,1,0,0,0,0,0


In [5]:
class Trainer(): #classe para facilitar o treinamento com o NB e SVC
    def __init__(self):
        self.data={}
        self.nb={}
        self.svc={}
        self.results_nb={}
        self.results_svc={}
        self.vc={}
        

    def pre_format(self,genre,data,not_data):
        data.df[genre]=1
        not_data.df[genre]=0
        dada_concat=Dataset(pd.concat([data.df,not_data.df]))
        return dada_concat
    
    def train(self,name,dataset,genre):  #Função para treinar
        #name = nome do treinamento(pode ser escolhido qualquer nome)
        #dataset = objeto da classe Dataset que recebera o treino
        #genre = genero a ser treinado
        
        self.vc[name]=DictVectorizer() #Transforma uma lista de valores em vetores para o SVC
        dataset.train_songs=self.vc[name].fit_transform(dataset.dict).toarray()
        #Usa o fit_transform do scikit-learn para organizar os dados de maneira para o fit(treino)
        
        nb=MultinomialNB() #Definindo o NaiveBayes
        svc=SVC() #Difinindo o SVC
        
        #O fit recebe Input(termos) e Output(1 e 0) em listas da seguinte maneira: nb.fit(Input,Output)
        self.nb[name]=nb.fit(dataset.train_songs,dataset.df[genre].tolist())
        self.svc[name]=svc.fit(dataset.train_songs,dataset.df[genre].tolist())
        return "The new dataset was trained and saved"
    
    def evaluate(self,name,new_songs): #metodo para salvar as musicas que a maquina classificou
        #name = nome do treino que você quer usar
        #new_songs = dataset de de musicas sem genero classificado
        
        new_songs.df=new_songs.df.reset_index(drop=True)
        new_songs.train_songs=self.vc[name].transform(new_songs.dict).toarray()

        sim_nb=[] #Lista de Sims(1) dados pelo NaiveBayes
        nao_nb=[] #Lista de Nãos(1) dados pelo NaiveBayes
        
        sim_svc=[] #Lista de Sims(1) dados pelo SVC
        nao_svc=[] #Lista de Nãos(1) dados pelo SVC
        
        nb_res=self.nb[name].predict(new_songs.train_songs)
        svc_res=self.svc[name].predict(new_songs.train_songs)
        
        for i in range(len(nb_res)):
            if nb_res[i] == 0:
                
                nao_nb.append(new_songs.df['song_title'][i]+' : '+new_songs.df['artist'][i])
            else:
                
                sim_nb.append(new_songs.df['song_title'][i]+' : '+new_songs.df['artist'][i])
            if svc_res[i] == 0:
                
                nao_svc.append(new_songs.df['song_title'][i]+' : '+new_songs.df['artist'][i])
            else:
                sim_svc.append(new_songs.df['song_title'][i]+' : '+new_songs.df['artist'][i])
                

        self.results_nb[name]={'sim':sim_nb,'nao':nao_nb}
        self.results_svc[name]={'sim':sim_svc,'nao':nao_svc}
        
        return "The result of the evaluation is now saved"
        
    
ash=Trainer() #Definindo o nosso treinador, com um nome apropriado

funk_train_df = pd.concat([funk_df.df,not_funk_df.df]) #juntando os dois Dataframes, note: Dataset.df é um Dataframe
ash.train("Funk",Dataset(funk_train_df),"funk")

rock_train_df = pd.concat([rock_df.df,not_rock_df.df])
ash.train("Rock",Dataset(rock_train_df),"rock")

eletronica_train_df = pd.concat([eletronica_df.df,not_eletronica_df.df])
ash.train("eletronica",Dataset(eletronica_train_df),"eletronica")

metal_train_df = pd.concat([metal_df.df,not_metal_df.df])
ash.train("metal",Dataset(metal_train_df),"metal")

sertanejo_train_df = pd.concat([sertanejo_df.df,not_sertanejo_df.df])
ash.train("sertanejo",Dataset(sertanejo_train_df),"sertanejo")

rap_train_df = pd.concat([rap_df.df,not_rap_df.df])
ash.train("rap",Dataset(rap_train_df),"rap")

'The new dataset was trained and saved'

<div id="dataset"></div>
### Classificando um Dataset
Agora que temos um treinamento para cada gênero, vamos usá-los para classificar uma Dataset que saibamos o conteúdo, como as duas playlists de Rock que baixamos no começo desse notebook. Com isso, vamos ver quantas músicas de Rock nosso classificador consegue indentificar nesse Dataset e quantas de Funk ele indentifica. Em teoria, se o classificador está funcionando, o número de música Rock deve ser maior que o de músicas Funk(que inclusive deveria ser 0, porém o modelo provavelmente apresentará falhas e acabará por indentificar algum Funk como Rock).

In [6]:
#Vamos utilizar a função evaluate, que recebe o genero a ser classificado e o Dataset
    
#Perceba que o nome inserido no Evaluate deve ser o do treino que você quer usar, criado anteriormente

test_rock = Dataset(df_spotipy_test)

ash.evaluate("Rock",test_rock)
#print(ash.results_nb["rock"])
#print(ash.results_svc["rock"])



'The result of the evaluation is now saved'

In [7]:
nb_musicas_rock = len(ash.results_nb['Rock']['sim']) #Vendo os resultados no NaiveBayes
nb_musicas_nrock = len(ash.results_nb['Rock']['nao'])

svc_musicas_rock = len(ash.results_svc['Rock']['sim']) #Vendo os resultados no SVC
svc_musicas_nrock = len(ash.results_svc['Rock']['nao'])

print(nb_musicas_rock,nb_musicas_nrock)
print(svc_musicas_rock,svc_musicas_nrock)
print("Podemos ver q nesse caso, o Naive Bayes se saiu melhor que o SVC, uma vez que marcou mais Rock em um Dataset apenas de Rock. Apesar de existirem diferentes métodos de comparação melhores que esse.")

68 32
48 52
Podemos ver q nesse caso, o Naive Bayes se saiu melhor que o SVC, uma vez que marcou mais Rock em um Dataset apenas de Rock. Apesar de existirem diferentes métodos de comparação melhores que esse.


In [8]:
#Agora vamos ver como eles classificam o mesmo Dataset para o gênero Funk
ash.evaluate("Funk",test_rock)
nb_musicas_funk = len(ash.results_nb['Funk']['sim']) #Vendo os resultados no NaiveBayes
nb_musicas_nfunk = len(ash.results_nb['Funk']['nao'])

svc_musicas_funk = len(ash.results_svc['Funk']['sim']) #Vendo os resultados no SVC
svc_musicas_nfunk = len(ash.results_svc['Funk']['nao'])

print(nb_musicas_funk,nb_musicas_nfunk)
print(svc_musicas_funk,svc_musicas_nfunk)
print("Podemos perceber que o classificador marcou consideravelmente menos músicas como Funk do que como Rock, o que mostra que apesar de não muito preciso, ele segue uma lógica que se comprova. Além disso, vimos mais uma vez o NB se saindo melhor que o SVC, ao marcar menos 2 erroneamente.")

36 64
38 62
Podemos perceber que o classificador marcou consideravelmente menos músicas como Funk do que como Rock, o que mostra que apesar de não muito preciso, ele segue uma lógica que se comprova. Além disso, vimos mais uma vez o NB se saindo melhor que o SVC, ao marcar menos 2 erroneamente.


<div id="versus"></div>
### Naive Bayes Vs SVC
Agora que já observamos os dois classificadores, vamos fazer uma comparação entre eles em diferentes cenários, um para cada gênero musical. Dessa forma podemos dizer qual método é melhor para nosso conjunto de dados e para nosso objetivo. Para isso, teremos de redefinir a classe Trainer, chamando ela de Trainer2, pois teremos de adicionar uma nova função.

In [9]:
#Vamos definir novamente a classe treinador, porém agora com algumas mudanças:

# Função test_this que esta um gênero com base em seu dataset e um dos outros datasets selecionados de outro gênero
# Função evaluate agora também salva um dataset de músicas com 1


class Trainer2():
        def __init__(self):
            self.data={}
            self.nb={}
            self.svc={}
            self.results_nb={}
            self.results_svc={}
            self.df_results_nb = {}
            self.df_results_svc = {}
            self.vc={}
        def pre_format(self,genre,data,not_data):
            data.df[genre]=1
            not_data.df[genre]=0
            dada_concat=Dataset(pd.concat([data.df,not_data.df]))
            return dada_concat
    
        def train(self,name,dataset,genre="ans"):
            self.vc[name]=DictVectorizer()
            dataset.train_songs=self.vc[name].fit_transform(dataset.dict).toarray()
            #name=nome da chave do dicionario que sera salvo o treinamento,
            #dataset=objeto da classe Dataset que recebera o treino
            
            nb=MultinomialNB()
            svc=SVC()
            if genre=="ans":
                self.nb[name]=nb.fit(dataset.train_songs,dataset.ans)
                self.svc[name]=svc.fit(dataset.train_songs,dataset.ans)
            else:
                self.nb[name]=nb.fit(dataset.train_songs,dataset.df[genre].tolist())
                self.svc[name]=svc.fit(dataset.train_songs,dataset.df[genre].tolist())
            return "The new dataset was trained and saved"
        
        def evaluate(self,name,new_songs): #metodo para salvar as musicas que a maquina classificou
            #name = nome do treino que você quer usar
            #new_songs = dataset de de musicas sem genero classificado
        
            new_songs.df=new_songs.df.reset_index(drop=True)
            new_songs.train_songs=self.vc[name].transform(new_songs.dict).toarray()
    
            sim_nb=[] #Lista de Sims(1) dados pelo NaiveBayes
            nao_nb=[] #Lista de Nãos(1) dados pelo NaiveBayes

            sim_svc=[] #Lista de Sims(1) dados pelo SVC
            nao_svc=[] #Lista de Nãos(1) dados pelo SVC
            
            nb_res=self.nb[name].predict(new_songs.train_songs)
            svc_res=self.svc[name].predict(new_songs.train_songs)
            framesnb = []
            framessvc = []
            for i in range(len(nb_res)):
                if nb_res[i] == 0:
                    
                    nao_nb.append(new_songs.df['song_title'][i]+' : '+new_songs.df['artist'][i])
                else:
                    sim_nb.append(new_songs.df['song_title'][i]+' : '+new_songs.df['artist'][i])
                    framesnb.append(new_songs.df.iloc[i:i+1])
                if svc_res[i] == 0:
                    
                    nao_svc.append(new_songs.df['song_title'][i]+' : '+new_songs.df['artist'][i])
                else:
                    sim_svc.append(new_songs.df['song_title'][i]+' : '+new_songs.df['artist'][i])
                    framessvc.append(new_songs.df.iloc[i:i+1])
    
            self.results_nb[name]={'sim':sim_nb,'nao':nao_nb}
            self.results_svc[name]={'sim':sim_svc,'nao':nao_svc}
            self.df_results_nb[name] = pd.concat(framesnb)
            self.df_results_svc[name] = pd.concat(framessvc)
            
            return "The result of the evaluation is now saved!"
        def check_score(self,name,new_songs,genre="ans"):
            #metodo para checar a eficacia do machine learning
            new_songs.train_songs=self.vc[name].transform(new_songs.dict).toarray()
            if genre=="ans":
                print("This is the SVC score: {}".format(self.svc[name].score(new_songs.train_songs,new_songs.ans)))
    
                print("This is the NB score: {}".format(self.nb[name].score(new_songs.train_songs,new_songs.ans)))
        
            else:
                svcscore = self.svc[name].score(new_songs.train_songs,new_songs.df[genre].tolist())
                print("This is the SVC score: {}".format(svcscore))
                nbscore = self.nb[name].score(new_songs.train_songs,new_songs.df[genre].tolist())
                print("This is the NB score: {}".format(nbscore))
                if svcscore > nbscore:
                      print("Para {} o SVC se saiu melhor".format(name))
                else:
                      print("Para {} o NB se saiu melhor".format(name))
        def test_this(self,name,genre):
            #name=nome que sera salvo o treinamento 
            #genre=o genero que sera treinado
            
            genre_2=genre
            genre_list=['funk','rock','eletronica','metal','sertanejo','rap']

            genre_name=genre+'_df.csv'
            genre=Dataset(genre_name)
            not_genre_name='not_'+genre_name
            not_genre=Dataset(not_genre_name)
            genre_all=self.pre_format(genre,genre,not_genre)

            self.train(name,genre_all,genre)
            test_name_df='test_'+genre_name
            test_genre_df=Dataset(test_name_df)
            x=[]
            for i in range(len(genre_list)):
                if genre_list[i]!=genre_2:
                    x.append(genre_list[i])
            random_genre=np.random.choice(x)
    
            
            test_not_name_df='test_'+random_genre+'_df.csv'
            test_genre_not_df=Dataset(test_not_name_df)

            test_genre_all=self.pre_format(genre,test_genre_df,test_genre_not_df)

            self.check_score(name,test_genre_all,genre)
            self.evaluate(name,test_genre_all)

In [10]:
misty=Trainer2() #Definindo nosso novo treinador

for i in lista_generos: #fazendo para cada item na lista de generos
    print(i)
    misty.test_this(i,i)

funk
This is the SVC score: 0.6571428571428571
This is the NB score: 0.6357142857142857
Para funk o SVC se saiu melhor
rock
This is the SVC score: 0.575
This is the NB score: 0.78
Para rock o NB se saiu melhor
eletronica
This is the SVC score: 0.6805555555555556
This is the NB score: 0.5277777777777778
Para eletronica o SVC se saiu melhor
metal
This is the SVC score: 0.723404255319149
This is the NB score: 0.5815602836879432
Para metal o SVC se saiu melhor
sertanejo
This is the SVC score: 0.533678756476684
This is the NB score: 0.8911917098445595
Para sertanejo o NB se saiu melhor
rap
This is the SVC score: 0.5
This is the NB score: 0.61
Para rap o NB se saiu melhor


In [15]:
def GiveMePlaylist(genre,mood,trainer=misty):
    gtrain_df =Dataset(pd.concat([pd.read_csv(genre+'_df.csv'),pd.read_csv('not_'+genre+'_df.csv')]))
    trainer.train(genre,gtrain_df,genre)
    mega_dataset = Dataset(pd.read_csv("giant_df.csv"))
    trainer.evaluate(genre,mega_dataset)
    filtered_ds = Dataset(trainer.df_results_nb[genre])
    
    mtrain_df = Dataset(pd.concat([pd.read_csv(mood+'_df.csv'),pd.read_csv('not_'+mood+'_df.csv')]))
    trainer.train(mood,mtrain_df,mood)
    trainer.evaluate(mood,filtered_ds)
    playlist = trainer.df_results_nb[mood]
    
    playlist = playlist.drop_duplicates()
    playlist = playlist.sample(frac=1)
    
    return playlist


In [16]:
test_playlist = GiveMePlaylist("metal","gym")
test_playlist

,Unnamed: 0,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,...,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,song_title,artist
337,41,0.000040,https://api.spotify.com/v1/audio-analysis/4ua0...,0.617,328253,0.617,4ua0IepBEISCWwF8dTJvcU,0.059800,8,0.2530,...,1,0.1220,127.996,4,https://api.spotify.com/v1/tracks/4ua0IepBEISC...,audio_features,spotify:track:4ua0IepBEISCWwF8dTJvcU,0.5170,Ghosts 'n' Stuff - feat. Rob Swire,deadmau5
21,36,0.039800,https://api.spotify.com/v1/audio-analysis/6oNZ...,0.294,232937,0.697,6oNZ0WxqptE7V0qpjtyJ4a,0.000110,6,0.1360,...,1,0.1280,160.566,4,https://api.spotify.com/v1/tracks/6oNZ0WxqptE7...,audio_features,spotify:track:6oNZ0WxqptE7V0qpjtyJ4a,0.1830,Falls (feat. Sasha Sloan),ODESZA
253,6,0.000052,https://api.spotify.com/v1/audio-analysis/7wvt...,0.371,251547,0.941,7wvtiQ3o8ELEmk2NjSSPpX,0.676000,5,0.0732,...,1,0.0602,120.499,4,https://api.spotify.com/v1/tracks/7wvtiQ3o8ELE...,audio_features,spotify:track:7wvtiQ3o8ELEmk2NjSSPpX,0.3860,Obstacle 1,Interpol
417,18,0.000021,https://api.spotify.com/v1/audio-analysis/25Df...,0.278,276093,0.975,25DfYJr3P66JuMDcOWtw8u,0.052700,10,0.6710,...,0,0.1930,152.635,4,https://api.spotify.com/v1/tracks/25DfYJr3P66J...,audio_features,spotify:track:25DfYJr3P66JuMDcOWtw8u,0.1870,Psychosocial - Live,Slipknot
88,42,0.009020,https://api.spotify.com/v1/audio-analysis/1WlM...,0.575,201231,0.920,1WlMjyGrxWEFAeNAxPrGdk,0.001870,11,0.1580,...,0,0.0835,129.979,4,https://api.spotify.com/v1/tracks/1WlMjyGrxWEF...,audio_features,spotify:track:1WlMjyGrxWEFAeNAxPrGdk,0.1030,Missing - Andrew Rayel & Mark Sixma Remix,Mark Sixma
182,50,0.026700,https://api.spotify.com/v1/audio-analysis/52wN...,0.469,186985,0.709,52wN20O1wRgaDZdJbR5sIW,0.000000,1,0.0944,...,1,0.0581,130.878,4,https://api.spotify.com/v1/tracks/52wN20O1wRga...,audio_features,spotify:track:52wN20O1wRgaDZdJbR5sIW,0.2890,Don't You Think (feat. Aanysa),MSTR ROGERS
192,79,0.001920,https://api.spotify.com/v1/audio-analysis/5qO7...,0.716,315000,0.855,5qO7eXO7lXa1UNZVri3kbw,0.000011,5,0.1850,...,1,0.0781,127.990,4,https://api.spotify.com/v1/tracks/5qO7eXO7lXa1...,audio_features,spotify:track:5qO7eXO7lXa1UNZVri3kbw,0.1920,Peace Of Mind - Myon & Shane 54 Summer Of Love...,Above & Beyond
174,29,0.028700,https://api.spotify.com/v1/audio-analysis/2Uou...,0.573,325714,0.929,2UouNFBaIdR7yGjUnZOWGO,0.053100,10,0.1070,...,0,0.0490,125.999,4,https://api.spotify.com/v1/tracks/2UouNFBaIdR7...,audio_features,spotify:track:2UouNFBaIdR7yGjUnZOWGO,0.4070,Believer (feat. CeeLo Green) - CYA Remix,CID
217,56,0.075400,https://api.spotify.com/v1/audio-analysis/6wmt...,0.690,198052,0.709,6wmtezDhKr47mnjoCsXDc0,0.285000,7,0.1530,...,1,0.0287,111.000,4,https://api.spotify.com/v1/tracks/6wmtezDhKr47...,audio_features,spotify:track:6wmtezDhKr47mnjoCsXDc0,0.4840,Congo,Bear Mountain
160,5,0.004700,https://api.spotify.com/v1/audio-analysis/6HZI...,0.637,185947,0.514,6HZILIRieu8S0iqY8kIKhj,0.000000,1,0.0940,...,1,0.3650,139.931,4,https://api.spotify.com/v1/tracks/6HZILIRieu8S...,audio_features,spotify:track:6HZILIRieu8S0iqY8kIKhj,0.4020,DNA.,Kendrick Lamar


### Resultados
Podemos perceber que o SVC se saiu melhor nos gêneros: Eletrônica e Metal, enquanto o NB se saiu melhor em Funk, Rock, Sertanejo e Rap. Isso demonstra que mesmo com Datasets com o mesmo número e tipo de variáveis pode-se obter melhores resultados trocando o método, dependendo do output desejado para a classificação. Porém, é possível observar que o NB levou uma considerável vantagem no total, talvez por conta dos indices, provavelmente, serem quase totalmente independentes.

#### O que falta:

- Acabar a conclusão
- Classificar os Moods
- Fazer Dataset resultante de uma filtragem Genero+Mood
- Revisar

##### Talvez:

- Salvar resultado de um Dataset em uma playlist no Spotify
- Fazer modelo de 0:5 generos para testar

<div id="melhorias"></div>
### Possíveis melhorias do modelo

<ul>
<li> Aumentar a base de dados para cada gênero deixaria os classificadores mais precisos</li>
<li> Aumentar quantidade de gêneros musicais, incluindo:pop,samba,música clássica,além de outros, como subgêneros por exemplo</li>
<li> Testar a classificação de Neighbours - Classificação que observa os "vizinhos" de um valor, ou seja, os pontos mais próximos e os relaciona. </li>
<li> Adaptar os dados e testar classificação por Rede Neurais - um tipo de algoritmo de ensino supervisionado que consegue trabalhar com funções não lineares</li>
</ul>